This notebook is designed to identify composers whose works might fall into the public domain within the European Union. It utilizes Wikipedia page views as a metric to filter out the most relevant entries.

This notebook is designed to identify composers whose works might fall into the public domain within the European Union. It utilizes Wikipedia page views as a metric to filter out the most relevant entries.

1. **Fetching a list of composers**: The first step involves fetching a list of composers from the Wikipedia page [List of 20th-century classical composers](https://en.wikipedia.org/wiki/List_of_20th-century_classical_composers).

The associated code cell performs the following tasks:

   1. **Get HTML content**: The `get_wiki_soup(url)` function fetches the HTML content of the provided URL using the `requests` library. It raises an exception if there's a server error. The HTML content is then parsed using BeautifulSoup.
   
   2. **Parse the HTML table**: The `parse_table(soup)` function finds the relevant table ("wikitable sortable") in the parsed HTML. If it can't find the table, it prints an error message. Otherwise, it goes through each row of the table (skipping the header row), retrieves the text content of each cell, and appends it to a list. It also finds the URL of each composer's Wikipedia page and appends it to the list. Each list of cell values is then converted into a dictionary (using the table headers as keys), and all these dictionaries are gathered into a list.
   
   3. **Create a DataFrame**: The URL of the Wikipedia page is defined, and the above functions are used to fetch and parse the page. If no errors occur, the list of dictionaries is converted into a pandas DataFrame and displayed.


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_wiki_soup(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # This will raise an exception if there's a server error
    except requests.RequestException as e:
        print(f"An error occurred while fetching the page: {e}")
        return None

    return BeautifulSoup(response.text, "html.parser")

def parse_table(soup):
    table = soup.find("table", {"class": "wikitable sortable"})
    if table is None:
        print("Could not find the table on the page")
        return None

    rows = table.find_all("tr")

    # Get the headers and add the 'URL' header
    headers = [header.get_text(strip=True) for header in rows[0].find_all("th")]
    headers.append('URL')

    # Get the data
    data = []
    for row in rows[1:]:  # Skip the first row because it's the headers
        values = [value.get_text(strip=True) for value in row.find_all("td")]

        # Get the URL of the composer's Wikipedia page
        link = row.find("td").find("a")
        if link is not None:
            values.append('https://en.wikipedia.org' + link.get('href'))
        else:
            values.append(None)

        data.append(dict(zip(headers, values)))

    return data

url = "https://en.wikipedia.org/wiki/List_of_20th-century_classical_composers"
soup = get_wiki_soup(url)
if soup is not None:
    data = parse_table(soup)
    if data is not None:
        # Convert the list of dictionaries into a DataFrame
        df = pd.DataFrame(data)
        display(df)


,Name,Year of birth,Year of death,Nationality,Notable 20th-century works,Remarks,URL
0,Charles Dancla,1817,1907,French,"Solo de concours no. 7, Op. 224",Romanticism,https://en.wikipedia.org/wiki/Charles_Dancla
1,Luigi Arditi,1822,1903,Italian,,,https://en.wikipedia.org/wiki/Luigi_Arditi
2,Theodor Kirchner,1823,1903,German,,,https://en.wikipedia.org/wiki/Theodor_Kirchner
3,Carl Reinecke,1824,1910,German,"Trio for piano, clarinet and horn in B♭, Op. 2...",Romanticism,https://en.wikipedia.org/wiki/Carl_Reinecke
4,Richard Hol,1825,1904,Dutch,Organ music,Romanticism,https://en.wikipedia.org/wiki/Richard_Hol
...,...,...,...,...,...,...,...
3362,Jimmy López,1978,,Peruvian,Arco de luz,,https://en.wikipedia.org/wiki/Jimmy_L%C3%B3pez
3363,Tarik O'Regan,1978,,British-American,,,https://en.wikipedia.org/wiki/Tarik_O%27Regan
3364,Mehdi Hosseini,1979,,Iranian,,,https://en.wikipedia.org/wiki/Mehdi_Hosseini
3365,Christophe Bertrand,1981,2010,French,,,https://en.wikipedia.org/wiki/Christophe_Bertrand


2. **Parsing the Wikipedia page views via the Wikimedia API**

This step involves fetching the page views for each composer's Wikipedia page. Note that if a page was deleted or created after your defined "end_date", an error might occur. These composers are still included in the DataFrame, but their pageviews are set to 0.

The associated code cell performs the following tasks:

   1. **Define the `get_pageviews` function**: This function takes a URL and a date range as input, extracts the title of the Wikipedia page from the URL, constructs the URL of the Wikimedia pageviews API, sends a GET request to that URL, and parses the response to extract the pageviews. If the URL is NaN, the function returns NaN. If the request fails for any reason, the function prints an error message and returns 0. The function also includes a delay to respect the API's rate limits.
   
   2. **Apply the function to the DataFrame**: The `get_pageviews` function is applied to the 'URL' column of the DataFrame, with the start and end dates set to define the date range for the pageviews. The results are stored in a new 'Pageviews' column in the DataFrame.
   
   3. **Sort the DataFrame**: The DataFrame is sorted by the 'Pageviews' column in descending order, so the composers with the most pageviews are at the top.
   
   4. **Display the DataFrame**: The sorted DataFrame is displayed to visualize the results.

Also, you need to replace `"YourActualUserAgent/1.0 (https://github.com/yourusername/yourrepository; mailto:youremail@example.com)"` with your actual user agent to properly use the Wikimedia API. The user agent should contain your application name, version, URL, and your email address.

In [2]:
from tqdm import tqdm
import requests
import pandas as pd
import numpy as np
import json
import time

def get_pageviews(url, start_date, end_date):
    if pd.isnull(url):  # Check if URL is NaN
        return np.nan

    # Extract the title from the URL
    title = url.split('/wiki/')[-1]

    base_url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/all-access/all-agents/"
    full_url = base_url + title + f"/monthly/{start_date}/{end_date}"  # adjust the date range as needed

    headers = {
        "User-Agent": "YourActualUserAgent/1.0 (https://github.com/yourusername/yourrepository; mailto:youremail@example.com)"
    }

    try:
        response = requests.get(full_url, headers=headers)
        response.raise_for_status()  # This will raise an exception for 4xx and 5xx status codes
        data = response.json()
    except (requests.RequestException, json.JSONDecodeError) as e:
        print(f"An error occurred while fetching pageviews for URL: {url}")
        print(f"Error: {e}")
        return 0

    # The 'items' field contains the pageview data
    pageviews = sum(item['views'] for item in data.get('items', []))

    # Delay to respect rate limits
    time.sleep(0.1)

    return pageviews


# Apply the get_pageviews function to the entire DataFrame
start_date = "20220101"
end_date = "20221231"
tqdm.pandas(desc="Fetching pageviews")
df['Pageviews'] = df['URL'].progress_apply(get_pageviews, start_date=start_date, end_date=end_date)


df_sorted = df.sort_values(by='Pageviews', ascending=False)

display(df_sorted)


Fetching pageviews:  15%|████████▉                                                  | 510/3367 [03:03<14:38,  3.25it/s]

An error occurred while fetching pageviews for URL: https://en.wikipedia.org/wiki/Eug%C3%A9nie-Victorine-Jeanne_Alombert
Error: 404 Client Error: Not Found for url: https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/all-access/all-agents/Eug%C3%A9nie-Victorine-Jeanne_Alombert/monthly/20220101/20221231


Fetching pageviews:  60%|██████████████████████████████████▊                       | 2018/3367 [12:13<07:00,  3.21it/s]

An error occurred while fetching pageviews for URL: https://en.wikipedia.org/wiki/Edward_Bland_(composer)
Error: 404 Client Error: Not Found for url: https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/all-access/all-agents/Edward_Bland_(composer)/monthly/20220101/20221231


Fetching pageviews:  68%|███████████████████████████████████████▎                  | 2280/3367 [13:47<05:32,  3.26it/s]

...

An error occurred while fetching pageviews for URL: https://en.wikipedia.org/w/index.php?title=Michel_Bosc&action=edit&redlink=1
Error: 404 Client Error: Not Found for url: https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/all-access/all-agents/https://en.wikipedia.org/w/index.php?title=Michel_Bosc&action=edit&redlink=1/monthly/20220101/20221231


Fetching pageviews:  99%|█████████████████████████████████████████████████████████▌| 3340/3367 [20:09<00:08,  3.15it/s]

An error occurred while fetching pageviews for URL: https://en.wikipedia.org/w/index.php?title=Fred_Jonny_Berg&action=edit&redlink=1
Error: 404 Client Error: Not Found for url: https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/all-access/all-agents/https://en.wikipedia.org/w/index.php?title=Fred_Jonny_Berg&action=edit&redlink=1/monthly/20220101/20221231


Fetching pageviews: 100%|██████████████████████████████████████████████████████████| 3367/3367 [20:19<00:00,  2.76it/s]


,Name,Year of birth,Year of death,Nationality,Notable 20th-century works,Remarks,URL,Pageviews
2643,Paul McCartney,1942,,English,Liverpool Oratorio;Standing Stone;Ecce Cor Meum,member ofThe BeatlesandWings,https://en.wikipedia.org/wiki/Paul_McCartney,4906941.0
2298,John Williams,1932,,American,"also composer of film scores,The Five Sacred T...",,https://en.wikipedia.org/wiki/John_Williams,1860855.0
2599,Frank Zappa,1940,1993,American,The Perfect Stranger;The Yellow Shark,,https://en.wikipedia.org/wiki/Frank_Zappa,1735503.0
2697,Vangelis,1943,,Greek,,electronic music,https://en.wikipedia.org/wiki/Vangelis,1011156.0
1727,Leonard Bernstein,1918,1990,American,West Side Story;Chichester Psalms;Candide;Sere...,jazz-influenced and pop-influence; also a comp...,https://en.wikipedia.org/wiki/Leonard_Bernstein,924195.0
...,...,...,...,...,...,...,...,...
2622,John Mitchell,1941,,American,,,None,NaN
3024,Ge Gan-ru,1954,,Chinese,,https://en.wikipedia.org/wiki/Ge_Gan-ru,NaN,NaN
3269,Paul Steenhuisen,1965,,Canadian,,https://en.wikipedia.org/wiki/Paul_Steenhuisen,NaN,NaN
3270,Mats Wendt,1965,,Eddan – The Invincible Sword of the Elf-Smith,,https://en.wikipedia.org/wiki/Mats_Wendt,NaN,NaN


3. **Data Cleaning and Type Conversion**

This step involves cleaning the DataFrame and converting certain columns to the appropriate data types.

The associated code cell performs the following tasks:

   1. **Remove rows with NaN values**: The DataFrame is filtered to remove rows where either the 'URL' or 'Pageviews' columns contain NaN values.
   
   2. **Convert columns to numeric**: The 'Year of birth' and 'Year of death' columns are converted to numeric values using `pd.to_numeric`. Any errors during the conversion (e.g., if a value cannot be converted to a number) are ignored, and the problematic values are converted to NaN.
   
   3. **Handle NaN values**: NaN values in the 'Year of birth' and 'Year of death' columns are replaced with a placeholder value (-1), the columns are then converted to integers, and the placeholder values are converted back to NaN. This is done to ensure that NaN values are handled correctly when the columns are converted to integer type.
   
   4. **Convert 'Year of birth' to float**: For consistency, the 'Year of birth' column is converted back to float type.
   
   5. **Display the DataFrame**: The cleaned and transformed DataFrame is displayed to visualize the results.


In [11]:
# Clean the parsed db, remove the rows where either 'URL' or 'Pageviews' is NaN
df_filtered = df_sorted.dropna(subset=['URL', 'Pageviews']).copy()

# Convert the 'Year of death' and 'Year of birth' columns to numeric
df_filtered['Year of birth'] = pd.to_numeric(df_filtered['Year of birth'], errors='coerce')
df_filtered['Year of death'] = pd.to_numeric(df_filtered['Year of death'], errors='coerce')

# Convert NaNs to a placeholder value (-1), convert to integers, and convert placeholders back to NaN
df_filtered['Year of birth'] = df_filtered['Year of birth'].fillna(-1).astype(int).replace(-1, np.nan)
df_filtered['Year of death'] = df_filtered['Year of death'].fillna(-1).astype(int).replace(-1, np.nan)

# Convert 'Year of birth' back to float for consistency
df_filtered['Year of birth'] = df_filtered['Year of birth'].astype(float)

display(df_filtered)


,Name,Year of birth,Year of death,Nationality,Notable 20th-century works,Remarks,URL,Pageviews
2643,Paul McCartney,1942.0,NaN,English,Liverpool Oratorio;Standing Stone;Ecce Cor Meum,member ofThe BeatlesandWings,https://en.wikipedia.org/wiki/Paul_McCartney,4906941.0
2298,John Williams,1932.0,NaN,American,"also composer of film scores,The Five Sacred T...",,https://en.wikipedia.org/wiki/John_Williams,1860855.0
2599,Frank Zappa,1940.0,1993.0,American,The Perfect Stranger;The Yellow Shark,,https://en.wikipedia.org/wiki/Frank_Zappa,1735503.0
2697,Vangelis,1943.0,NaN,Greek,,electronic music,https://en.wikipedia.org/wiki/Vangelis,1011156.0
1727,Leonard Bernstein,1918.0,1990.0,American,West Side Story;Chichester Psalms;Candide;Sere...,jazz-influenced and pop-influence; also a comp...,https://en.wikipedia.org/wiki/Leonard_Bernstein,924195.0
...,...,...,...,...,...,...,...,...
508,Eugénie-Victorine-Jeanne Alombert,1874.0,1964.0,French,Menuet champêtre,,https://en.wikipedia.org/wiki/Eug%C3%A9nie-Vic...,0.0
3232,Michel Bosc[fr],1963.0,NaN,French,,,https://en.wikipedia.org/w/index.php?title=Mic...,0.0
2278,M. William Karlins,1932.0,2005.0,American,Works forSaxophone; works forDouble Bass[vague],,https://en.wikipedia.org/w/index.php?title=M._...,0.0
2655,Clive Strutt,1942.0,NaN,English,The Tragedy of Man,neoromantic,https://en.wikipedia.org/w/index.php?title=Cli...,0.0


4. **Filtering Composers by Year of Death**

This step involves filtering out the composers based on their year of death. We focus on those who passed away between 1945 and 1955.

The associated code cell performs the following tasks:

   1. **Apply filter conditions**: The DataFrame is filtered to include only those composers whose 'Year of death' is between 1945 and 1955 (inclusive).
   
   2. **Display the DataFrame**: The filtered DataFrame is displayed to visualize the results.


In [12]:
# Filter the DataFrame
df_public_domain = df_filtered[(df_filtered['Year of death'] >= 1945) & (df_filtered['Year of death'] <= 1955)]

display(df_public_domain)


,Name,Year of birth,Year of death,Nationality,Notable 20th-century works,Remarks,URL,Pageviews
903,Sergei Prokofiev,1891.0,1953.0,Russian,Romeo and Juliet;The Love for Three Oranges;Cl...,"neoclassicism,Romanticism",https://en.wikipedia.org/wiki/Sergei_Prokofiev,396736.0
524,Arnold Schoenberg,1874.0,1951.0,Austrian,Gurre-Lieder;Verklärte Nacht;Variations for Or...,"Romanticism, laterexpressionismandserialism; f...",https://en.wikipedia.org/wiki/Arnold_Schoenberg,320754.0
349,Richard Strauss,1864.0,1949.0,German,An Alpine Symphony;Der Rosenkavalier; Oboe Con...,Romanticism,https://en.wikipedia.org/wiki/Richard_Strauss,319356.0
656,Béla Bartók,1881.0,1945.0,Hungarian,Concerto for Orchestra;six string quartets;The...,"neoclassicism, folk-influenced",https://en.wikipedia.org/wiki/B%C3%A9la_Bart%C...,255906.0
958,Ivor Novello,1893.0,1951.0,Welsh,,,https://en.wikipedia.org/wiki/Ivor_Novello,215507.0
...,...,...,...,...,...,...,...,...
1133,Niels Clemmensen,1900.0,1950.0,Danish,,,https://en.wikipedia.org/wiki/Niels_Clemmensen,653.0
251,Teresa Tanco Cordovez de Herrera,1859.0,1946.0,Colombian,,,https://en.wikipedia.org/wiki/Teresa_Tanco_Cor...,614.0
894,Thomas Griselle,1891.0,1955.0,American,,,https://en.wikipedia.org/wiki/Thomas_Griselle,577.0
209,George Strong,1856.0,1948.0,American,,,https://en.wikipedia.org/wiki/George_Strong_(c...,535.0


5. **Parsing IMSLP for Corresponding Pages**

This step involves creating the corresponding IMSLP (International Music Score Library Project) URL for each composer and checking if a page exists at that URL.

The associated code cell performs the following tasks:

   1. **Define the `create_imslp_link` function**: This function takes a composer's name, splits it into parts, and creates the corresponding IMSLP URL.
   
   2. **Define the `check_page_exists` function**: This function sends a GET request to a given URL and checks if the response status code is 200, indicating that a page exists at that URL. It includes a delay to prevent overloading the server.
   
   3. **Create a copy of the DataFrame**: The DataFrame is copied to avoid modifying the original data.
   
   4. **Create the IMSLP URLs**: The `create_imslp_link` function is applied to the 'Name' column of the DataFrame, and the resulting IMSLP URLs are stored in a new 'IMSLP_URL' column.
   
   5. **Check if the IMSLP pages exist**: The `check_page_exists` function is applied to the 'IMSLP_URL' column of the DataFrame, and the results are stored in a new 'IMSLP_Exists' column. The `tqdm.pandas()` function is used to display a progress bar for the operation.
   
   6. **Display the DataFrame**: The DataFrame, now including the IMSLP URLs and whether a page exists at each URL, is displayed.


In [14]:
import pandas as pd
import requests
import time
from tqdm import tqdm

def create_imslp_link(composer):
    name_parts = composer.split(' ')

    if len(name_parts) == 1:
        imslp_name = name_parts[0]
    else:
        imslp_name = "{},_{}".format(name_parts[-1], '_'.join(name_parts[:-1]))

    return f"https://imslp.org/wiki/Category:{imslp_name}"

def check_page_exists(url):
    time.sleep(0.1)  # pause to prevent overloading server
    response = requests.get(url)
    return response.status_code == 200

# Replace 'composers.tsv' with the path to your tsv file if you want to load oyur df from tsv
# df_imslp = pd.read_csv('composers.tsv', sep='\t')
df_imslp = df_public_domain.copy()

df_imslp['IMSLP_URL'] = df_imslp['Name'].apply(create_imslp_link)

# Apply tqdm to the dataframe
tqdm.pandas()

df_imslp['IMSLP_Exists'] = df_imslp['IMSLP_URL'].progress_apply(check_page_exists)

display(df_imslp)


100%|████████████████████████████████████████████████████████████████████████████████| 207/207 [01:51<00:00,  1.86it/s]


,Name,Year of birth,Year of death,Nationality,Notable 20th-century works,Remarks,URL,Pageviews,IMSLP_URL,IMSLP_Exists
903,Sergei Prokofiev,1891.0,1953.0,Russian,Romeo and Juliet;The Love for Three Oranges;Cl...,"neoclassicism,Romanticism",https://en.wikipedia.org/wiki/Sergei_Prokofiev,396736.0,"https://imslp.org/wiki/Category:Prokofiev,_Sergei",True
524,Arnold Schoenberg,1874.0,1951.0,Austrian,Gurre-Lieder;Verklärte Nacht;Variations for Or...,"Romanticism, laterexpressionismandserialism; f...",https://en.wikipedia.org/wiki/Arnold_Schoenberg,320754.0,"https://imslp.org/wiki/Category:Schoenberg,_Ar...",True
349,Richard Strauss,1864.0,1949.0,German,An Alpine Symphony;Der Rosenkavalier; Oboe Con...,Romanticism,https://en.wikipedia.org/wiki/Richard_Strauss,319356.0,"https://imslp.org/wiki/Category:Strauss,_Richard",True
656,Béla Bartók,1881.0,1945.0,Hungarian,Concerto for Orchestra;six string quartets;The...,"neoclassicism, folk-influenced",https://en.wikipedia.org/wiki/B%C3%A9la_Bart%C...,255906.0,"https://imslp.org/wiki/Category:Bartók,_Béla",True
958,Ivor Novello,1893.0,1951.0,Welsh,,,https://en.wikipedia.org/wiki/Ivor_Novello,215507.0,"https://imslp.org/wiki/Category:Novello,_Ivor",True
...,...,...,...,...,...,...,...,...,...,...
1133,Niels Clemmensen,1900.0,1950.0,Danish,,,https://en.wikipedia.org/wiki/Niels_Clemmensen,653.0,"https://imslp.org/wiki/Category:Clemmensen,_Niels",False
251,Teresa Tanco Cordovez de Herrera,1859.0,1946.0,Colombian,,,https://en.wikipedia.org/wiki/Teresa_Tanco_Cor...,614.0,"https://imslp.org/wiki/Category:Herrera,_Teres...",False
894,Thomas Griselle,1891.0,1955.0,American,,,https://en.wikipedia.org/wiki/Thomas_Griselle,577.0,"https://imslp.org/wiki/Category:Griselle,_Thomas",True
209,George Strong,1856.0,1948.0,American,,,https://en.wikipedia.org/wiki/George_Strong_(c...,535.0,"https://imslp.org/wiki/Category:Strong,_George",False


6. **Filtering Rows Where IMSLP Page Exists and Displaying with Clickable Links**

This step involves filtering out the rows where the IMSLP page exists (i.e., 'IMSLP_Exists' is True) and displaying the DataFrame as HTML with clickable links.

The associated code cell performs the following tasks:

   1. **Filter the DataFrame**: The DataFrame is filtered to include only those rows where 'IMSLP_Exists' is True. The filtered DataFrame is copied to avoid modifying the original data.
   
   2. **Create clickable links**: The 'IMSLP_URL' and 'URL' columns are transformed into clickable HTML links.
   
   3. **Display the DataFrame as HTML**: The DataFrame is converted to an HTML table with `to_html`, and this HTML is displayed. The `escape=False` argument is used to render the HTML links correctly. Only the first 100 rows of the DataFrame are displayed.

In [19]:
df_imslp_exists = df_imslp[df_imslp['IMSLP_Exists'] == True].copy()

df_imslp_exists['IMSLP_URL'] = df_imslp_exists['IMSLP_URL'].apply(lambda x: '<a href="{}">{}</a>'.format(x,x))
df_imslp_exists['URL'] = df_imslp_exists['URL'].apply(lambda x: '<a href="{}">{}</a>'.format(x,x))

from IPython.display import HTML

HTML(df_imslp_exists.head(100).to_html(escape=False))


,Name,Year of birth,Year of death,Nationality,Notable 20th-century works,Remarks,URL,Pageviews,IMSLP_URL,IMSLP_Exists
903,Sergei Prokofiev,1891.0,1953.0,Russian,Romeo and Juliet;The Love for Three Oranges;Classical Symphony;Symphony No. 5andNo. 6;Violin Concerto No. 1andNo. 2;Piano Concerto No. 2andNo. 3;Alexander Nevsky;Lieutenant Kijé;Peter and the Wolf,"neoclassicism,Romanticism",https://en.wikipedia.org/wiki/Sergei_Prokofiev,396736.0,"https://imslp.org/wiki/Category:Prokofiev,_Sergei",True
524,Arnold Schoenberg,1874.0,1951.0,Austrian,Gurre-Lieder;Verklärte Nacht;Variations for Orchestra;Pierrot Lunaire;Five Pieces for Orchestra;Violin Concerto,"Romanticism, laterexpressionismandserialism; founder of theSecond Viennese School",https://en.wikipedia.org/wiki/Arnold_Schoenberg,320754.0,"https://imslp.org/wiki/Category:Schoenberg,_Arnold",True
349,Richard Strauss,1864.0,1949.0,German,An Alpine Symphony;Der Rosenkavalier; Oboe Concerto;Salome;Metamorphosen,Romanticism,https://en.wikipedia.org/wiki/Richard_Strauss,319356.0,"https://imslp.org/wiki/Category:Strauss,_Richard",True
656,Béla Bartók,1881.0,1945.0,Hungarian,"Concerto for Orchestra;six string quartets;The Miraculous Mandarin;Piano Concerto No. 1;Piano Concerto No. 2;Piano Concerto No. 3;Music for Strings, Percussion and Celesta;Sonata for Two Pianos and Percussion","neoclassicism, folk-influenced",https://en.wikipedia.org/wiki/B%C3%A9la_Bart%C3%B3k,255906.0,"https://imslp.org/wiki/Category:Bartók,_Béla",True
958,Ivor Novello,1893.0,1951.0,Welsh,,,https://en.wikipedia.org/wiki/Ivor_Novello,215507.0,"https://imslp.org/wiki/Category:Novello,_Ivor",True
820,Florence Price,1887.0,1953.0,American,,,https://en.wikipedia.org/wiki/Florence_Price,149155.0,"https://imslp.org/wiki/Category:Price,_Florence",True
1161,Kurt Weill,1900.0,1950.0,German/American,"2 symphonies; String Quartets; ""Die Dreigroschenoper"";Aufstieg und Fall der Stadt Mahagonny; ""Mahagonny-Songspiel""; several Broadway musicals",neoclassical,https://en.wikipedia.org/wiki/Kurt_Weill,145638.0,"https://imslp.org/wiki/Category:Weill,_Kurt",True
519,Charles Ives,1874.0,1954.0,American,The Unanswered Question;Central Park in the Dark;Variations on America;Three Places in New England;Concord Sonata,"avant-garde, folk-influenced",https://en.wikipedia.org/wiki/Charles_Ives,113714.0,"https://imslp.org/wiki/Category:Ives,_Charles",True
735,Anton Webern,1883.0,1945.0,Austrian,Symphony; Six Pieces for Orchestra,"twelve-tone technique,serialism,polystylism; pupil ofArnold Schoenberg",https://en.wikipedia.org/wiki/Anton_Webern,93929.0,"https://imslp.org/wiki/Category:Webern,_Anton",True
794,Wilhelm Furtwängler,1886.0,1954.0,German,"Symphony No. 1 in B minor, Symphony No. 2 in E minor, Symphony No. 3 in C♯ minor, Piano Quintet, Religöser Hymnus, Te Deum for Choir and Orchestra","neoclassicism, folk-influenced, twelve-tone technique",https://en.wikipedia.org/wiki/Wilhelm_Furtw%C3%A4ngler,87943.0,"https://imslp.org/wiki/Category:Furtwängler,_Wilhelm",True


7. **Saving the DataFrame as a TSV File**

This step involves saving your desired DataFrame as a TSV (Tab Separated Values) file for future reference or use.

The associated code cell performs the following tasks:

   1. **Define the DataFrame to be saved**: The DataFrame to be saved is defined. In this case, it is `df_sorted`, which is the DataFrame obtained after sorting by page views. Throughout the notebook, we worked with several DataFrames: `df` (initial DataFrame from the Wikipedia page), `df_sorted` (sorted by page views), `df_filtered` (cleaned and with numeric columns), `df_public_domain` (filtered by year of death), `df_imslp` (with IMSLP URLs and existence checks), and `df_imslp_exists` (filtered to include only rows where an IMSLP page exists). You can change `df_sorted` to any of these DataFrames depending on which one you want to save.
   
   2. **Save the DataFrame as a TSV file**: The DataFrame is saved as a TSV file using `to_csv` with `sep='\t'`. The `index=False` argument is used to exclude the DataFrame's index from the file. If the DataFrame is saved successfully, a success message is printed with the current working directory and the filename. If an error occurs during the saving process, an error message is printed.


In [ ]:
import os
from datetime import datetime

# Define the DataFrame to be saved
df_to_save = df_sorted

# save the DataFrame as a TSV file
filename = 'composers.tsv'
try:
    df_to_save.to_csv(filename, sep='\t', index=False)
    print(f"Data saved successfully to {os.getcwd()}\\{filename} at {datetime.now()}.")
except:
    print("Error: data could not be saved.")
